# Cleaning And Preprocessing - Monster India

In [1]:
!pip install nltk

  Using cached nltk-3.6.2-py3-none-any.whl (1.5 MB)
You should consider upgrading via the '/Users/admin/Projects/Msc/Data Mining/mining-env/bin/python3 -m pip install --upgrade pip' command.


In [25]:
import pandas as pd
import re
from datetime import date
import numpy as np
import math
from nltk import everygrams

In [3]:
CSV_PATH = '../Scrapping/monster-india-jobs.csv'

df = pd.read_csv(CSV_PATH)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16800 entries, 0 to 16799
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       16800 non-null  int64 
 1   timestamp        16800 non-null  object
 2   location         16793 non-null  object
 3   job_title        16800 non-null  object
 4   company_name     16800 non-null  object
 5   package          16800 non-null  object
 6   experience       16800 non-null  object
 7   job_description  16800 non-null  object
 8   skills           16230 non-null  object
dtypes: int64(1), object(8)
memory usage: 1.2+ MB


In [4]:
df.sample(2)

,Unnamed: 0,timestamp,location,job_title,company_name,package,experience,job_description,skills
3402,3402,Posted: 30+ days ago,Pune,Sr. Salesforce Developer,iQuest Management Consultants Private Limited,"5,00,000-13,00,000 INR Per Annum",3-8 Years,Roles and Responsibilities\n\n * Should have ...,"metadata,force.com,Salesforce,Sprint Planning,..."
688,688,Posted: 11 days ago,Bengaluru / Bangalore,Senior Technical Writer,M/s It Edge Pro Services,"14,00,000-18,00,000 INR Per Annum",7-10 Years,Looking for Senior Technical Writer with 7-9 y...,"DITA XML authoring& publishing,xml tools,Netwo..."


## Data Cleaning

Most of the columns have data in a format which is not suitable for analysis.

###### Below, we shall clean the column data one by one.

### Remove 'Unnamed: 0 columns'

In [5]:
del df['Unnamed: 0']

In [6]:
'Unnamed: 0' in df.columns # check if columns still exists

False

### timestamp column

Notice that the data comes in forms of: 
+ `Posted: N days ago` 
+ `Posted: an hour ago` 
+ `Posted: N hours ago`
+ `Posted: a day ago`
+ `Posted: a month ago`

Also, the data was downloaded at 9:00 am Approx. on 1st July 2021.

### Treatment
+ We care only about the date and not the exact time (hour, seconds) on which the job post was made
+ Hence, every timestamp that has hours, will have Date as 01/06/2021.
+ For the Edge case of Month 'AGO', we consider that as 01/05/2021.

#### Algorithm 
+ Hence, we are left to extact the N from `N days ago` 
+ Subtract `N` from `31` with month as `05 (may)`. 
+ The Postings with `N hours ago` falls in `1st July`

For sake of scalability we'll keep month number and days as CONSTANTs so that in future, code can be ran as it is by simply updating the month and days of past month.

In [7]:
df.timestamp.unique()

array(['Posted: 17 days ago', 'Posted: 18 days ago',
       'Posted: 19 days ago', 'Posted: 30+ days ago',
       'Posted: 11 days ago', 'Posted: 7 days ago', 'Posted: a month ago',
       'Posted: 20 days ago', 'Posted: 16 days ago',
       'Posted: 25 days ago', 'Posted: 10 days ago',
       'Posted: 22 days ago', 'Posted: 23 days ago',
       'Posted: 14 days ago', 'Posted: 12 days ago',
       'Posted: 13 days ago', 'Posted: a day ago', 'Posted: 2 days ago',
       'Posted: an hour ago', 'Posted: 8 hours ago',
       'Posted: 9 hours ago', 'Posted: 11 hours ago',
       'Posted: 12 hours ago', 'Posted: 13 hours ago',
       'Posted: 14 hours ago', 'Posted: 15 hours ago',
       'Posted: 16 hours ago', 'Posted: 17 hours ago',
       'Posted: 18 hours ago', 'Posted: 19 hours ago',
       'Posted: 20 hours ago', 'Posted: 21 hours ago',
       'Posted: 8 days ago', 'Posted: 9 days ago', 'Posted: 21 days ago',
       'Posted: 3 days ago', 'Posted: 4 days ago', 'Posted: 5 days ago',
    

In [8]:
print('''Example of Regex Based Extraction''')

test_str = 'Posted: 17 days ago'
# find the occurrance of Digits i.e. `\d', '+' for more then one digit 
re_extract_num = re.compile('\d+')
print(re_extract_num.findall(test_str))

Example of Regex Based Extraction
['17']


In [9]:
LAST_MONTH = 5
LAST_MONTH_DAYS = 31
YEAR = 2021
PUBLICATION_DAY = 1

def extract_days_before_N(input_string):
    '''
        return timestamp
        
        if the input_string is in the form of Posted: N days ago, it returns timestamp wtr to past month
        for input_string in form of `hours ago`, the date of publication is used.
    '''
    re_extract_num = re.compile('\d+')
    finds = re_extract_num.findall(input_string)
    if(len(finds) > 0):
        num = int(finds[0])
        exact_day = LAST_MONTH_DAYS - num
        exact_date = date(YEAR, LAST_MONTH, exact_day)
        return int(exact_date.strftime("%s"))
    else:
        publication_date = date(YEAR, LAST_MONTH + 1, PUBLICATION_DAY)
        return int(publication_date.strftime("%s"))

# Map string to integer(timestamp)
df.timestamp = df.timestamp.apply(extract_days_before_N)

In [10]:
# Result
df.timestamp.unique()

array([1620930600, 1620844200, 1620757800, 1619807400, 1621449000,
       1621794600, 1622485800, 1620671400, 1621017000, 1620239400,
       1621535400, 1620498600, 1620412200, 1621189800, 1621362600,
       1621276200, 1622226600, 1621708200, 1621621800, 1621103400,
       1620585000, 1622140200, 1622053800, 1621967400, 1620325800,
       1621881000])

### Packages

+ In the extracted Dataset, `package` is in categorical format.
+ There are about `1250` categories of Packages
+ This makes it easier for classification based machine learning algorithm to work
+ However, It would be helpful to extract `minimum`, `maximum` and `mean` salary from the given package

So we Keep the package (categorical) variable as it as, and create 3 more features:
+ minimum_salary
+ maximum_salary
+ mean_salary

In [11]:
print(df.package.unique())
print()
print(f'There are about `{len(df.package.unique())}` categories of Packages')

['60,000-3,00,000 INR Per Annum' '50,000-2,00,000 INR Per Annum'
 '50,000-3,00,000 INR Per Annum' ... '4,20,000-8,80,000 INR Per Annum'
 '3,00,000-12,00,000 INR Per Annum' '28,00,000-35,00,000 INR Per Annum']

There are about `1250` categories of Packages


In [12]:
print('Verify if package categories have consistent form & Check if all the values are in the form of N-N INR Per Annum')
print(f"All Values in package feature have Per Annum")
print()
print(f'df.package.str.contains("Per Annum").all() = `{df.package.str.contains("Per Annum").all()}`')

Verify if package categories have consistent form & Check if all the values are in the form of N-N INR Per Annum
All Values in package feature have Per Annum

df.package.str.contains("Per Annum").all() = `True`


In [13]:
test='60,000-3,00,000 INR Per Annum'
split = test.split(' ')[0]
print(f"Split the sentence on space and pick the first element i.e. {split}")
min_sal, max_sal = split.split('-')
print(f"Again split the item wtr to '-' we get {min_sal} and {max_sal}")
integer_min_sal = int(min_sal.replace(',',''))
integer_max_sal = int(max_sal.replace(',',''))
print(f"Substitute '-' with '', we get {integer_min_sal} and {integer_max_sal}")

Split the sentence on space and pick the first element i.e. 60,000-3,00,000
Again split the item wtr to '-' we get 60,000 and 3,00,000
Substitute '-' with '', we get 60000 and 300000


In [14]:

def convert_package_to_min_salaries(package):
    hyphened_salary = package.split(' ')[0]
    sal_1, sal_2 = hyphened_salary.split('-')
    integer_sal_1 = int(sal_1.replace(',',''))
    integer_sal_2 = int(sal_2.replace(',',''))
    min_sal = min(integer_sal_1, integer_sal_2)
    return min_sal


def convert_package_to_max_salaries(package):
    hyphened_salary = package.split(' ')[0]
    sal_1, sal_2 = hyphened_salary.split('-')
    integer_sal_1 = int(sal_1.replace(',',''))
    integer_sal_2 = int(sal_2.replace(',',''))
    max_sam = max(integer_sal_1, integer_sal_2)
    return max_sal


def convert_package_to_mean_salaries(package):
    hyphened_salary = package.split(' ')[0]
    sal_1, sal_2 = hyphened_salary.split('-')
    integer_sal_1 = int(sal_1.replace(',',''))
    integer_sal_2 = int(sal_2.replace(',',''))
    mean_sal = np.average([integer_sal_1, integer_sal_2])
    return mean_sal


min_salary_feature = df.package.apply(convert_package_to_min_salaries)
max_salary_feature = df.package.apply(convert_package_to_max_salaries)
mean_salary_feature = df.package.apply(convert_package_to_mean_salaries)

#### Create New Features:
+ min_salary
+ max_salary
+ mean_salary

In [15]:
df['min_salary'] = min_salary_feature
df['max_salary'] = max_salary_feature
df['mean_salary'] = mean_salary_feature

In [16]:
df.columns

Index(['timestamp', 'location', 'job_title', 'company_name', 'package',
       'experience', 'job_description', 'skills', 'min_salary', 'max_salary',
       'mean_salary'],
      dtype='object')

### Experience

#### Values are in the form of `N-M Years` or `Fresher`

#### Algorithm

##### Method 1
+ Split by space and grab the first element. we get `N-M`
+ Split by '-' to get 'N' and 'M'
+ Convert to integer and compute, min,max, mean experience

##### Method 2
+ Use Regex to grab the integers
+ use `min()` and `max()`

We'll also see which is faster method.

In [17]:
df.experience.unique()

array(['4-9 Years', '0-5 Years', '1-7 Years', '0-10 Years', '3-8 Years',
       '5-10 Years', '6-10 Years', '3-7 Years', '3-6 Years',
       '11-16 Years', '1-8 Years', '4-8 Years', '0-4 Years', '2-3 Years',
       '3-4 Years', '5-6 Years', '4-10 Years', '1-6 Years',
       'Not Specified', '1-5 Years', '1-3 Years', '8-10 Years',
       '2-12 Years', '1-4 Years', '2-4 Years', '2-5 Years', '0-2 Years',
       '8-12 Years', '7-12 Years', '6-16 Years', '2-8 Years', '2-7 Years',
       '25-31 Years', '5-8 Years', '5-9 Years', '2-6 Years',
       '10-15 Years', '0-3 Years', '8-14 Years', '3-5 Years', '4-6 Years',
       '6-15 Years', '4-12 Years', '13-20 Years', '6-11 Years',
       '12-22 Years', '10-20 Years', '8-15 Years', '4-7 Years',
       '0-1 Years', '3-10 Years', '7-14 Years', '15-20 Years',
       '6-13 Years', '6-9 Years', '9-14 Years', '3-12 Years',
       '10-16 Years', '9-16 Years', '7-11 Years', '1-2 Years',
       '2-20 Years', '12-17 Years', '6-12 Years', '5-12 Years',
    

In [18]:
# Method 1
def extract_min_experience(input_string):
    re_exp = re.compile("\d+")
    findall = re_exp.findall(input_string)
    if(len(findall) == 2):
        a, b = int(findall[0]), int(findall[1])
        min_exp = min(a,b)
        return min_exp
    return 0

# Method 2
def extract_min_experience_split(input_string: str):
    if(input_string.find("Years") != -1):
        exp_range = input_string.split(' ')[0]
        exp_1, exp_2 = exp_range.split('-')
        min_exp = min(int(exp_1), int(exp_2))
        return min_exp
    return 0

def extract_max_experience(input_string: str):
    if(input_string.find("Years") != -1):
        exp_range = input_string.split(' ')[0]
        exp_1, exp_2 = exp_range.split('-')
        min_exp = max(int(exp_1), int(exp_2))
        return min_exp
    return 0

def extract_mean_experience(input_string: str):
    if(input_string.find("Years") != -1):
        exp_range = input_string.split(' ')[0]
        exp_1, exp_2 = exp_range.split('-')
        min_exp = np.average([int(exp_1), int(exp_2)])
        return min_exp
    return 0

##### Comparing Regex Approach with standard method split
##### This is rather a suprising result where using regex takes more time compared to using Naive split based extraction

In [152]:
%%timeit
df.experience.apply(extract_min_experience)

42.1 ms ± 1.23 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [151]:
%%timeit
df.experience.apply(extract_min_experience_split)

22 ms ± 726 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


##### Creating Features
+ min_experience
+ max_experience
+ mean_experience

In [19]:
df['min_experience'] = df.experience.apply(extract_min_experience)
df['max_experience'] = df.experience.apply(extract_max_experience)
df['mean_experience'] = df.experience.apply(extract_mean_experience)

In [20]:
df.columns

Index(['timestamp', 'location', 'job_title', 'company_name', 'package',
       'experience', 'job_description', 'skills', 'min_salary', 'max_salary',
       'mean_salary', 'min_experience', 'max_experience', 'mean_experience'],
      dtype='object')

In [21]:
df.sample(2)

,timestamp,location,job_title,company_name,package,experience,job_description,skills,min_salary,max_salary,mean_salary,min_experience,max_experience,mean_experience
7586,1620412200,"Mumbai, India",Senior Cyber Security Manager,ANI Calls India Private Limited,"9,00,000-18,00,000 INR Per Annum",5-10 Years,Job Description :\n\nThe candidate should be a...,Senior Cyber Security Manager,900000,"3,00,000",1350000.0,5,10,7.5
6737,1620930600,"Mumbai, Mumbai City, Navi Mumbai, Thane",required accounts executive /senior accountant,Company Name Confidential,"2,20,000-5,30,000 INR Per Annum",2-7 Years,Position: Accounts Executive/senior accountant...,"Accountant,Account Assistant,Account Executive...",220000,"3,00,000",375000.0,2,7,4.5


### Skills

```
Sample Data (before cleaning)

>> array(['Experienced', 'Contract Staffing', 'It Recruiter,C2H', ...,
       'Xylem', 'Cyber Sec Principal consultant', 'Azure Systems/DevOps'],
      dtype=object)
```
Also notice that skills are comma separated

#### Procedure for standarizing skills:
+ Lowercase the skills
+ Replace special characters with '' (excluding '/' and ',')
+ Handle NaN

### One hot encoding

Its very challenging to analyse 'skills' feature which is in, comma separated string form. Its possible to run queries which identify occurrance of a skill in the string, but this won't get us far. 

It'll help answering queries like How many Jobs require "Java" but fail to answer questions in form, "Which is the most required programming tool". The former question could be easily solved if the skills are in OHE form, i.e. their is different column for each skill, say java, and values are binary number, 0 representing absense of skill, while 1 denoting opposite.


### Problem in performing One Hot Encoding 

The skills string is very inconsistant as it contains irrelvant, spelling mistakes, similar skill but different wordings and so on. 

Below we have tried to apply OHE over first 100 rows ONLY. Look at the number of features(columns) it resulted, 432! 
This only gets worse as we'll see, where for entire dataset, it resulted in almost 15k skills feature! needless to say, this very computationally expensive, as well as most of the information it provides may not be important to us, for eg. there are about 3k skills which have just single occurrance. 

Below we display the difficulties and proceed to..
+ Apply N Grams when individual skill's string results in greater then 2 length. eg. 'information security software engineer', will be broken down into: 'information security' and 'software engineer'. Which make much more sense.
+ Consider skills which are represented at least N times. (so we exclude skills which are very sparse)
+ Now we have reduced the possible number of skills to couple of hundreds, altough it might sound counter-intuitive, we hand select each feature, reducing it to high quality data.
+ Apply OHE on them, by using apply method repeatedly by traversing each skill and using string matching.


Note: we try to get the skills strng in comma separated form by removing special chars, to apply pandas get_dummies method, which works on the basis of single separator.

In [169]:
df.skills[:100].str.get_dummies(sep=",")

,"""Ajax""]","""CSS""","""Git""","""HTML""","""IT Staffing""","""IT contract Staffing""","""Javascript""","""Recuriter""","""TDD frameworks""","""mySql""",...,senior customer support executive,sourcer,spring mvc,st.,tech support,testing method,uk process,wardrobes,web chat,zoho crm
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:

def clean_skills(input_string):
    '''Perform: Lowercase, Remove Special Chars ,Perform N grams over skills larger then 2 words'''
    if(type(input_string) == str):
        lower = input_string.lower()
        substituted = re.sub(r"[^,\w\s/]", '', lower)
        output = set()
        for i in substituted.split(','):
            if len(i.split(' ')) <= 2:
                output.add(i)
            else:
                ngrams = everygrams(i.strip().split(' '),1, 2)
                for gram in ngrams:
                    output.add(' '.join(gram))
        
        return ','.join(output)
# Eg. of skills string with special chars '[]' and '""'
test_str = '["Laravel","mySql","HTML","CSS","Javascript","Ajax/Jax", information security software engineer]'
clean_skills(test_str)

'html,mysql,css,ajax/jax,security,information,security software,software,software engineer,information security,laravel,engineer,javascript'

In [34]:
df.skills = df.skills.apply(clean_skills)

In [130]:
df.skills.unique()

array(['experienced', 'contract staffing', 'it recruiter,c2h', ...,
       'xylem',
       'principal consultant,sec,consultant,cyber sec,cyber,sec principal,principal',
       'azure systems/devops'], dtype=object)

#### Finding the distribution of skills.

##### Stage 1
- We perform split over each individual skills which are in form of 'java, j2ee, aws lambda'
- store each of them in dictionary with key as individual skill i.e. 'java' and value as counter.
- we get output in form of `{java: 845, python: 500 ....}`
- here the key represents the skill and value represents the count.

##### Stage 2 - group skills wtr to count i.e. `{ 845: ['java', 'scala'], .... }`

##### Stage 3 - group skill count wtr to number of skills with that count i.e. `{ 845: 2 }`

Now we can see the distribution of skills where we can answer, how many skills are repeated only once in entire dataset.

In [131]:
skills_super_set = dict()

def agg_skills_counter(input_string):
    if(type(input_string) == str):
        for i in input_string.split(','):
            skills_super_set[i] = skills_super_set.get(i, 0) + 1
    return None

_ = df.skills.agg(agg_skills_counter)

In [38]:
print(f'Number of skills = {len(skills_super_set)}')

Length of all skills 15219


In [115]:
count_list = {}
for skill in skills_super_set:
    if(skill):
        value = skills_super_set[skill]
        temp = list()
        count_list.get(value, temp).append(skill)
        if(len(temp) > 0):
            count_list[value] = temp

In [114]:
count_list

{'82': ['experienced',
  'change management',
  'es6',
  'costing',
  'store',
  'embedded systems',
  'sales cloud',
  'it hiring',
  'bulk hiring',
  'hr recruitment',
  'java full',
  'asp net',
  'front'],
 '12': ['contract staffing',
  'automobile manager',
  'routers',
  'spring mvc',
  'interviewing',
  'design manager',
  'senior customer',
  'bpo process',
  'customer associate',
  'exp',
  'healthcare',
  'email support',
  'performance testing',
  'service management',
  'wan',
  'google cloud',
  'mm',
  '/ lead',
  'logistics',
  'advanced',
  'report',
  'java/j2ee',
  'ml',
  'junior',
  'salesforce lead',
  'salesforce crm',
  'unit testing',
  'siebel',
  'magento',
  'lamp',
  'apps',
  'knowledge of',
  'problem solving',
  'english',
  'engineer with',
  'senior data',
  'account manager',
  'relationship',
  'ariba',
  'risk management',
  'smo',
  'developing',
  'analyst ',
  'direct sales',
  'azure ad',
  'sales officer',
  'performance tuning',
  'plm',
  'pro

In [127]:
n_skills = {}
for c in count_list:
    n_skills[c] = len(count_list[c])

In [128]:
n_skills # read as, 82 skill categories have appeared only 13 times in entire dataset.

{82: 13,
 12: 78,
 298: 2,
 14: 40,
 114: 2,
 36: 5,
 90: 3,
 64: 2,
 26: 18,
 13: 54,
 41: 3,
 589: 1,
 699: 1,
 341: 1,
 19: 30,
 1: 8779,
 6: 234,
 583: 1,
 56: 3,
 15: 44,
 5: 498,
 113: 5,
 125: 1,
 120: 3,
 133: 1,
 1666: 1,
 2: 1994,
 552: 1,
 4: 861,
 107: 2,
 208: 1,
 77: 19,
 512: 1,
 9: 110,
 3: 1001,
 7: 171,
 320: 1,
 51: 3,
 368: 1,
 338: 1,
 11: 76,
 17: 32,
 18: 16,
 75: 59,
 150: 4,
 224: 3,
 223: 9,
 81: 10,
 152: 3,
 165: 1,
 149: 6,
 129: 3,
 227: 1,
 485: 1,
 331: 2,
 21: 22,
 186: 1,
 105: 2,
 102: 2,
 629: 1,
 89: 3,
 23: 23,
 96: 4,
 49: 6,
 46: 4,
 8: 140,
 43: 4,
 119: 2,
 644: 1,
 115: 2,
 554: 1,
 156: 2,
 88: 5,
 439: 1,
 87: 6,
 570: 1,
 628: 1,
 99: 6,
 97: 2,
 103: 3,
 116: 2,
 229: 2,
 217: 1,
 415: 2,
 44: 4,
 76: 42,
 85: 7,
 209: 2,
 10: 87,
 283: 1,
 174: 2,
 142: 2,
 29: 13,
 166: 2,
 100: 3,
 39: 10,
 108: 2,
 164: 1,
 376: 1,
 226: 2,
 121: 4,
 118: 1,
 193: 2,
 16: 27,
 302: 1,
 232: 1,
 127: 4,
 407: 1,
 214: 1,
 147: 2,
 98: 5,
 264: 1,
 478: 

In [133]:
# We sort the skill with respect to how many times they appeared.
{k: v for k, v in sorted(n_skills.items(), key=lambda item: item[1], reverse=True)}

# so, we can see that skills that have appeared only once in entire datasets amount to 8779!!

{1: 8779,
 2: 1994,
 3: 1001,
 4: 861,
 5: 498,
 6: 234,
 7: 171,
 8: 140,
 9: 110,
 74: 101,
 10: 87,
 12: 78,
 11: 76,
 75: 59,
 13: 54,
 15: 44,
 76: 42,
 14: 40,
 17: 32,
 19: 30,
 16: 27,
 23: 23,
 79: 23,
 21: 22,
 80: 21,
 77: 19,
 26: 18,
 31: 18,
 27: 18,
 25: 17,
 18: 16,
 22: 15,
 82: 13,
 29: 13,
 20: 13,
 84: 13,
 28: 12,
 24: 11,
 81: 10,
 39: 10,
 223: 9,
 83: 9,
 78: 8,
 92: 8,
 85: 7,
 33: 7,
 34: 7,
 86: 7,
 93: 7,
 149: 6,
 49: 6,
 87: 6,
 99: 6,
 59: 6,
 60: 6,
 36: 5,
 113: 5,
 88: 5,
 98: 5,
 95: 5,
 38: 5,
 112: 5,
 35: 5,
 110: 5,
 150: 4,
 96: 4,
 46: 4,
 43: 4,
 44: 4,
 121: 4,
 127: 4,
 153: 4,
 54: 4,
 37: 4,
 101: 4,
 91: 4,
 30: 4,
 45: 4,
 311: 4,
 90: 3,
 41: 3,
 56: 3,
 120: 3,
 51: 3,
 224: 3,
 152: 3,
 129: 3,
 89: 3,
 103: 3,
 100: 3,
 47: 3,
 161: 3,
 155: 3,
 32: 3,
 151: 3,
 168: 3,
 50: 3,
 48: 3,
 61: 3,
 148: 3,
 205: 3,
 94: 3,
 298: 2,
 114: 2,
 64: 2,
 107: 2,
 331: 2,
 105: 2,
 102: 2,
 119: 2,
 115: 2,
 156: 2,
 97: 2,
 116: 2,
 229: 2,
 4

#### Hence we consider the skills which have appeared at least 100 times in entire dataset

In [143]:
[x for x in filter(lambda x: len(x)>1 and skills_super_set[x] > 50 ,skills_super_set)]

['experienced',
 'it recruiter',
 'production',
 'mechanical engineering',
 'project manager',
 'mechanical engineer',
 'bpo',
 'voice',
 'support',
 'support engineer',
 'desktop',
 'desktop support',
 'maintenance',
 'networking',
 'engineer',
 'diploma',
 'engineering',
 'design',
 'flooring',
 'civil engineer',
 'civil',
 'in',
 'architect',
 'and',
 'wardrobes',
 'dealers',
 'building products',
 'interiors',
 'products',
 'channel sales',
 'building',
 'furniture',
 'prefabricated building',
 'garden furniture',
 'laminates',
 'panels glass',
 'kitchens',
 'prefabricated',
 'furnishings',
 'outdoor furniture',
 'html',
 'css',
 'python',
 'javascript',
 'angular',
 'aspnet',
 'jquery',
 'web api',
 'cnet',
 'net core',
 'kubernetes',
 'azure',
 'paas',
 'aws',
 'iaas',
 'devops',
 'as',
 'cloud',
 'plsql',
 'oracle',
 'sql',
 'us it',
 'us staffing',
 'us',
 'technical recruiter',
 'recruiter',
 'sourcing',
 'it',
 'mvc',
 'web content',
 'content writer',
 'postgresql',
 'spring

##### Hand Select Features

In [145]:
hand_selected_features = [
 'experienced',
 'it recruiter',
 'project manager',
 'mechanical engineer',
 'support engineer',
 'desktop support',
 'maintenance',
 'networking',
 'engineer',
 'diploma',
 'design',
 'civil engineer',
 'html',
 'css',
 'python',
 'javascript',
 'angular',
 'aspnet',
 'jquery',
 'web api',
 'cnet',
 'net core',
 'kubernetes',
 'azure',
 'paas',
 'aws',
 'iaas',
 'devops',
 'as',
 'cloud',
 'plsql',
 'oracle',
 'sql',
 'us',
 'technical recruiter',
 'it',
 'mvc',
 'content writer',
 'postgresql',
 'spring boot',
 'itsm',
 'database',
 'customer service',
 'call center',
 'fresher',
 'non voice',
 'backend',
 'data entry',
 'back office',
 'non it',
 'communication skills',
 'process',
 'hr',
 'consultant',
 'scala',
 'btech',
 'quality',
 'autocad',
 'stack',
 'full stack',
 'stack developer',
 'developer',
 'full',
 'mern stack',
 'mean stack',
 'business analyst',
 'executive',
 'talent acquisition',
 'staffing',
 'it staffing',
 'css3',
 'php',
 'mysql',
 'development',
 'java',
 'web',
 'html5',
 'ajax',
 'lead generation',
 'sap',
 'engineer electrical',
 'electrical engineer',
 'be',
 'quality engineer',
 'production engineer',
 'electronics engineer',
 'diploma mechanical',
 'graduation',
 'lead',
 'application',
 'angularjs',
 'microservices',
 'reactjs',
 'nodejs',
 'job',
 'hardware',
 'embedded',
 'software',
 'software engineer',
 'support executive',
 'international bpo',
 'technical support',
 'customer support',
 'senior',
 'bca',
 'rpa',
 'developer ',
 'os installation',
 'troubleshooting',
 'laptop',
 'laptop support',
 'coordinator',
 'data engineer',
 'screening',
 'nonit recruiter',
 'scheduling',
 'hrss',
 'git',
 'core java',
 'w2',
 'bsc',
 'golang',
 'linux',
 'automation testing',
 'software testing',
 'manual testing',
 'sql server',
 'calling',
 'training',
 'nosql',
 'microsoft',
 'service',
 'enterprise',
 'web developer',
 'bootstrap',
 'net developer',
 'ngrx',
 'es6',
 'ms',
 'server',
 'ms sql',
 'embedded c',
 'scripting languages',
 'salesforce',
 'sprint planning',
 'lightning',
 'forcecom',
 'customization',
 'triggers',
 'apex',
 'subject matter expertise',
 'programming',
 'architectures',
 'cloud certification',
 'gcp',
 'application architecture',
 'expertise',
 'mongodb',
 'web components',
 'salesforce developer',
 'building codes',
 'building safety',
 'gulf',
 'facility management',
 'occupational health',
 'electrical engineering',
 'gcc',
 'safety standards',
 'statutory compliances',
 'oracle scm',
 'ghp',
 'haccp',
 'gmp',
 'glp',
 'sales',
 'marketing',
 'press shop',
 'inprocess quality',
 'sheet metal',
 'assembly line',
 'assembly shop',
 'general accounting',
 'bank reconciliation',
 'tally erp',
 'bom',
 'solid works',
 'costing',
 'stock keeping',
 'node js',
 'react js',
 'go lang',
 'redux',
 'architectural design',
 'software architecture',
 'system architecture',
 'enterprise architecture',
 'platform development',
 'oracle procurement',
 'hibernate',
 'spring',
 'docker',
 'ansible',
 'embedded systems',
 'multithreaded',
 'c programming',
 'freertos',
 'oracle fusion',
 'tech',
 'placement officer',
 'placement executive',
 'placement coordination',
 'placement coordinator',
 'skill development',
 'ui',
 'ui path',
 'tech architect',
 'rpa technical',
 'blueprism ui',
 'technical architect',
 'service cloud',
 'cassandra dba',
 'redis dba',
 'dba',
 'mongo dba',
 'database admin',
 'postgresql database',
 'administrator',
 'database administrator',
 'admin',
 'undergraduate',
 'hiring',
 'hiring undergraduate',
 'undergraduate experienced',
 'firmware',
 'kernel',
 'entry level',
 'mass mailing',
 'mass sms',
 'bfsi',
 'mass hiring',
 'it hiring',
 'level hiring',
 'entry',
 'sales hiring',
 'bulk hiring',
 'job postings',
 'pharma',
 'level',
 'basic hiring',
 'data structure',
 'algorithm',
 'protocol development',
 'protocol developer',
 'routing protocol',
 'hadoop',
 'spark',
 'perl',
 'data',
 'azure data',
 'mass',
 'oracle functional',
 'finance',
 'ebs',
 'technical writer',
 'management ',
 'social',
 'database management',
 'social media',
 'management',
 'media',
 'cassandra',
 'opentsdb talend',
 'paas',
 'tools',
 'mulesoft',
 'cassandra etl',
 'etl',
 'ci/cd ',
 'iaas',
 'saas',
 'ssis',
 'pl/sql',
 'cnet sql',
 'influxdb',
 'clerk',
 'office assistant',
 'computer operator',
 'office executive',
 'back operations',
 'office operations',
 'fresher trainee',
 'solid',
 'subcontrol',
 'subcontrol system',
 'control system',
 'equipment drawings',
 'pi diagram',
 'flow control',
 'imcc',
 'instrumentation',
 'diagram',
 'gas analysers',
 'automation equipment',
 'cement',
 'vvfc drives',
 'system',
 'distributors',
 'windows',
 'project sale',
 'dotnet',
 'hr recruitment',
 'hr executive',
 'recruitment executive',
 'oracle database',
 'dba oracle',
 'oracle dba',
 'service tax',
 'general ledger',
 'budgeting',
 'accounts',
 'finance executive',
 'statutory deductions',
 'utility bills',
 'gst',
 'payable ',
 'cash flow',
 'finance manager',
 'financial statements',
 'accounts payable',
 'tds',
 'accounts manager',
 'payable',
 'receivable',
 'tally',
 'jira',
 'sw development/',
 'c',
 'polarion',
 'embedded sw',
 'svn',
 'accounts assistant',
 'accounts executive',
 'finance',
 'auditing',
 'accounting',
 'manager',
 'generalist management',
 'recruitment management',
 'generalist',
 'compensation',
 'human resource',
 'infrastucture',
 'itil',
 'change',
 'bmc remedy',
 'datacenter',
 'bmc',
 'problem management',
 'monitoring tools',
 'global',
 'it infrastructure',
 'construction',
 'incident manager',
 'mass communication',
 'manualtest',
 'automation',
 'functional testing',
 'logistics manager',
 'stores logistics',
 'logistics coordinator',
 'customs documentation',
 'logistics executive',
 'care executive',
 'angular js',
 'java developer',
 'j2ee',
 'java full',
 'oracle pl/sql',
 'accountant',
 'counselling',
 'monitoring',
 'supervisor',
 'employee relations',
 'computer science',
 'mca',
 'scripting',
 'tcl',
 'inventory',
 'supply',
 'mrp',
 'wip',
 'supply chain',
 'oracle inventory',
 'bills',
 'chain',
 'chain management',
 'asp net',
 'network engineer',
 'network administrator',
 'hardware networking',
 'hardware and',
 'system administrator',
 'server programming',
 'it sales',
 'sales manager',
 'assistant',
 'inside sales',
 'development manager',
 'assistant sales',
 'api',
 'xml',
 'analytics',
 'testing',
 'associate',
 'security',
 'with',
 'iot',
 'react',
 'agile',
 'hana',
 'big data',
 'specialist',
 'cyber',
 'project',
 'system engineer',
 'analyst',
 'ibm',
 'integration',
 'solution',
 'cyber security',
 'network',
 'workday',
 'systems',
 'solution architect',
 'bi',
 'mobile',
 'fusion',
 'qa',
 'associate engineer',
 'test',
 'programmer',
 'excel',
 'front office',
 'front desk',
 'frontend',
 'solutions',
 'cashier',
 'office executiv',
 'freshers',
 'scrum',
 'electronics']

#### Apply One Hot Encoding

In [166]:
def skill_filter(skill):
    def filter(input_string):
        if type(input_string) == str:
            skill_list = input_string.split(',')
            for s in skill_list:
                if s == skill:
                    return True
        return False
    return filter

# we could match by string like df[df.skills.str.contains('java').fillna(False)], but this would match java with in javascript!
df[df.skills.apply(skill_filter('java'))].sample(2)

,timestamp,location,job_title,company_name,package,experience,job_description,skills,min_salary,max_salary,mean_salary,min_experience,max_experience,mean_experience
5844,1619807400,Pune,"Hiring for Java Developer"" - Pune (Immediate J...",Saksoft Limited,0-0 INR Per Annum,3-13 Years,Greeting form Saksoft Limited.\n \n \n Hiring ...,"java,spring boot,web services,spring,java 8",0,"3,00,000",0.0,3,13,8.0
2798,1619807400,"Bengaluru / Bangalore, Chennai",ReactJS Full Stack Developer | 4-6 LPA | 3+ Years,Disha Recruitment Services,"4,00,000-6,00,000 INR Per Annum",3-7 Years,"Key Skills 3+ Years : ReactJS, PHP / CSS, MySQ...","server programming,programming,mysql,java,sql,...",400000,"3,00,000",500000.0,3,7,5.0


In [167]:
# Will create 400+ features for each tech
for skill in hand_selected_features:
    df[skill] = df.skills.apply(skill_filter(skill)).fillna(False)

In [217]:
# Manually Adding few additional skills
df['android'] = df.skills.str.contains('android').fillna(False)
df['ios'] = df.skills.str.contains('ios').fillna(False)
df['game'] = df.skills.str.contains('game').fillna(False)
df['machine learning'] = df.skills.str.contains('android').fillna(False)
df['data analyst'] = df.skills.str.contains('data analyst').fillna(False)
df['artificial intelligence'] = df.skills.str.contains('artificial intelligence').fillna(False)

In [218]:
df.columns

Index(['timestamp', 'location', 'job_title', 'company_name', 'package',
       'experience', 'job_description', 'skills', 'min_salary', 'max_salary',
       ...
       'office executiv', 'freshers', 'scrum', 'electronics', 'android', 'ios',
       'game', 'machine learning', 'data analyst', 'artificial intelligence'],
      dtype='object', length=468)

In [220]:
df.to_csv('monster-india-clean.csv')